# Generative Adveserial Networks

Generative Adveserial Networks (GAN) are type of neural network architecture that is originally designed to generate samples which are close to orignal sample space. As the name suggests it is Generative network, which means that there is no supervised learning required. The general structure of GANs is provided in figure below. The figure shows two different networks present in the framework. They are commonly named Generator Network and Discriminator Network.

The two networks could be categorized as adversaries of each other. The basic purpose of the Generator Network is to learn the data distribution which is almost equivalent to original data distribution. The Generator Network starts with the noisy distribution and starts to learn from it. The network learns from the feedback provided by the Discriminator Network. The Discriminator could generally be any function that is differentiable. The network is trained on the sample latent data that have dimensions less than or equal to that of original data. Using this strategy allows the Generator to learn so generic features and represent the sample space in less dimesions. This ease the way to create further samples which are not in the original space. Inputs to the generator are randomly sampled from the model’s prior over the latent variables.

The Discriminator Network is provided with two types of data; the original data and the noisy data (i.e: data from Generator Network). Discriminator Network could be thought of as a binary classifier. The task of this binary classifier is to distinguish original data from generated data. It outputs 1 when the classifier thinks that the data is from original distribution and it outputs 0 if the classifier thinks that the data is from Generator Network.

Overall framework of GAN could be seen as a two player game where Generator Network is trying to get past the Discriminator Network by making it believe that the generated sample is from the original distibution. On the other hand Discriminator Network tries it best to not be fooled by Generative Network. In Ideal case If both models have sufficient capacity, then the Nash equilibrium of this game corresponds to the G(z) being drawn from the same distribution as the training data, and D(x) = 1/2 for all x.

**The GANs have mostly similar to VAEs, the most salient difference is that, if relying on standard backprop, VAEs cannot have discrete variables at the input to the generator, while GANs cannot have discrete variables at the output of the generator. The GAN approximation is subject to the failures of supervised learning: overfitting and underfitting.**

## Training and Cost Functions

There could be two types of training, for the first part of the training the Discriminator is fed with the original data. The data has half fake and half original data samples. The goal of Descriminator is to identify as much original data samples and output 1. 

In second part, generator is fed with latent variables from model's prior. The Generator tries to create fake data items and those data items are passed to Discriminator Network. In this part, the goal of Generator is to keep the output near to 1 whereas Discriminator tries to keep output near 0.

There are two cost functions for this framework. One for Generative Network and one for Discriminator Network. The commonly used cost function for Discriminator Network is modified cross entrophy, where it is expected that there are two types on inputs.

$$C_{D} = -\frac{1}{2}\mathbb{E}_{x\sim p(data)}log(D(x)) -\frac{1}{2}\mathbb{E}_{z}log(1- D(G(z)))$$

As could be seen from equation above, it maximize the data samples with are from original data and minimize the samples which are generated from Generator Network.

By training the discriminator we get the estimated ration of the two models

$$\frac{p_{data}(x)}{p_{model}(x)}$$

This is key approximation technqiue that sets GANs apart form variational autoencoders and Boltzmann machines.


As for Generator Network a simple cost function would be negative of the Discriminator cost function 

$$C_{G} = - C_{D}$$

The advantage of this approch is that we would only need to calculate the value function for adversary game. We can than find minmax function for one cost function and that would optimize the whole framework.

$$\theta_{G} = \text{argmin}_{\theta^{(G)}}  \text{max}_{\theta^{(D)}}C_{G}$$

However, there is also a big disadvantage of this cost function. Imagine a scenerio where discriminator successfully rejects generator samples with high confidence, in this case the generators's gradients vanishes. This could be intuitively seen in the above equation of cost function for discriminator. When D(G(z)) is 0 or near zero than the cost is almost zero, and taking derivative with zero initial value will vanish the gradients before reaching the input layer of neural network.

To overcome this problem we use a different cost function for Generator network.

$$C_{G} = -\frac{1}{2}\mathbb{E}_{z}log(D(G(z)))$$

Intuiviely this means that the generator maximizes the logprobability
of the discriminator being mistaken.

### DCGAN

Deep, Convolutional GAN (DCGAN), is the common architecture used in modern applications. DCGANs were initially used to generate high resolution images. They perform this task in one iteration where as prior to DCGANs, LAPGANs were used which required generation of image multiples in one iteration.

The best practicies for training DCGAN can be found on the following side https://github.com/soumith/ganhacks



#### Get packges

In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

/Users/daniyalusmani/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Define initial variables

In [2]:
# Input shape
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

optimizer = Adam(0.0002, 0.5)

The paper of Radford et al. gives some hints about what is a good DCGAN architecture :

* Replace any pooling layers with strided convolutions (discriminator) and fractional-strided convolutions (generator). In other words, when the generator needs to increase the spatial dimension of the representation, it uses transposed convolution with a stride greater than 1

* Use Batch Normalization in both the generator (except at the output layer) and the discriminator (except at the input layer)
* Remove fully connected hidden layers for deeper architectures
* Use ReLU activation in generator for all layers except for the output
* Use LeakyReLU activation in the discriminator for all layers

For creation of generator, we have used Transposed conv2D layer with batch normalization applied

<img src="./files/dc-gan-flow.png" 
alt="IMAGE ALT TEXT HERE" border="10" />

In [3]:
def buildGenerator():

    model = Sequential()

    model.add(Dense(128 * 28 * 28, activation="relu", input_dim=latent_dim))
    model.add(Reshape((28, 28, 128)))
    model.add(Conv2DTranspose(128, kernel_size=3, padding="same"))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2DTranspose(64, kernel_size=3, padding="same"))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2DTranspose(channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

The discriminator is
generated using the Conv2D layer, with activation function set to LeakyReLU and batch Normalization is applied

In [4]:
def buildDiscriminator():

    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [5]:
# Build and compile the discriminator
discriminator = buildDiscriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

# Build the generator
generator = buildGenerator()

# The generator takes noise as input and generates imgs
z = Input(shape=(100,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
valid = discriminator(img)

# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
__________

In [8]:
def save_imgs(epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, latent_dim))
        gen_imgs = generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()

In [9]:
def train(epochs, batch_size=128, save_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Sample noise and generate a batch of new images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)   
    
        # Train the discriminator (real classified as ones and generated as zeros)
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        # Train the generator (wants discriminator to mistake images as real)
        g_loss = combined.train_on_batch(noise, valid)

        # Plot the progress
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            save_imgs(epoch)

In [11]:
train(epochs=1001, batch_size=32, save_interval=100)

/Users/daniyalusmani/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.750284, acc.: 50.00%] [G loss: 0.700566]
1 [D loss: 0.422570, acc.: 82.81%] [G loss: 1.015291]
2 [D loss: 0.509268, acc.: 70.31%] [G loss: 0.697980]
3 [D loss: 0.512294, acc.: 75.00%] [G loss: 0.756566]
4 [D loss: 0.702772, acc.: 60.94%] [G loss: 0.894716]
5 [D loss: 0.616657, acc.: 67.19%] [G loss: 0.880327]
6 [D loss: 0.584337, acc.: 76.56%] [G loss: 0.709457]
7 [D loss: 0.940414, acc.: 46.88%] [G loss: 0.564096]
8 [D loss: 0.754347, acc.: 60.94%] [G loss: 1.446072]
9 [D loss: 0.946122, acc.: 40.62%] [G loss: 0.418355]
10 [D loss: 0.626583, acc.: 64.06%] [G loss: 0.948578]
11 [D loss: 0.582154, acc.: 71.88%] [G loss: 1.345667]
12 [D loss: 0.708845, acc.: 57.81%] [G loss: 0.627676]
13 [D loss: 0.708231, acc.: 57.81%] [G loss: 0.771116]
14 [D loss: 0.465311, acc.: 81.25%] [G loss: 1.057621]
15 [D loss: 0.520009, acc.: 71.88%] [G loss: 1.522194]
16 [D loss: 0.820395, acc.: 51.56%] [G loss: 1.256114]
17 [D loss: 0.788396, acc.: 50.00%] [G loss: 1.679573]
18 [D loss: 1.211594

149 [D loss: 0.682671, acc.: 65.62%] [G loss: 1.045471]
150 [D loss: 0.694060, acc.: 60.94%] [G loss: 0.960235]
151 [D loss: 0.607461, acc.: 67.19%] [G loss: 0.811827]
152 [D loss: 0.695574, acc.: 60.94%] [G loss: 0.861982]
153 [D loss: 0.723855, acc.: 62.50%] [G loss: 0.790639]
154 [D loss: 0.879635, acc.: 54.69%] [G loss: 0.821891]
155 [D loss: 0.577335, acc.: 67.19%] [G loss: 1.349121]
156 [D loss: 0.633316, acc.: 65.62%] [G loss: 1.048312]
157 [D loss: 0.718648, acc.: 62.50%] [G loss: 0.697412]
158 [D loss: 0.705160, acc.: 56.25%] [G loss: 1.033154]
159 [D loss: 0.863859, acc.: 46.88%] [G loss: 1.154215]
160 [D loss: 0.562787, acc.: 71.88%] [G loss: 0.957323]
161 [D loss: 0.807699, acc.: 51.56%] [G loss: 1.143473]
162 [D loss: 0.706562, acc.: 60.94%] [G loss: 1.216813]
163 [D loss: 0.582858, acc.: 64.06%] [G loss: 1.471809]
164 [D loss: 0.600518, acc.: 65.62%] [G loss: 0.951359]
165 [D loss: 0.711669, acc.: 62.50%] [G loss: 1.264914]
166 [D loss: 0.435961, acc.: 81.25%] [G loss: 1.

296 [D loss: 0.537458, acc.: 73.44%] [G loss: 1.390195]
297 [D loss: 0.510333, acc.: 75.00%] [G loss: 1.226361]
298 [D loss: 0.606100, acc.: 71.88%] [G loss: 1.353870]
299 [D loss: 0.752768, acc.: 59.38%] [G loss: 1.009928]
300 [D loss: 0.618196, acc.: 67.19%] [G loss: 1.237538]
301 [D loss: 0.590253, acc.: 68.75%] [G loss: 1.432524]
302 [D loss: 0.703005, acc.: 59.38%] [G loss: 1.172559]
303 [D loss: 0.634137, acc.: 70.31%] [G loss: 1.266512]
304 [D loss: 0.693260, acc.: 57.81%] [G loss: 0.851710]
305 [D loss: 0.681150, acc.: 64.06%] [G loss: 1.284671]
306 [D loss: 0.611970, acc.: 64.06%] [G loss: 0.777686]
307 [D loss: 0.517174, acc.: 70.31%] [G loss: 1.320845]
308 [D loss: 0.582522, acc.: 68.75%] [G loss: 1.165201]
309 [D loss: 0.475131, acc.: 81.25%] [G loss: 1.317626]
310 [D loss: 0.803466, acc.: 53.12%] [G loss: 1.064551]
311 [D loss: 0.612799, acc.: 73.44%] [G loss: 0.862805]
312 [D loss: 0.667424, acc.: 65.62%] [G loss: 1.316735]
313 [D loss: 0.522669, acc.: 75.00%] [G loss: 1.

443 [D loss: 0.649835, acc.: 67.19%] [G loss: 1.515904]
444 [D loss: 0.681306, acc.: 64.06%] [G loss: 1.306240]
445 [D loss: 0.775018, acc.: 50.00%] [G loss: 1.239265]
446 [D loss: 0.623996, acc.: 60.94%] [G loss: 1.219285]
447 [D loss: 0.738185, acc.: 46.88%] [G loss: 1.389983]
448 [D loss: 0.587523, acc.: 62.50%] [G loss: 1.055153]
449 [D loss: 0.565991, acc.: 75.00%] [G loss: 1.406944]
450 [D loss: 0.671331, acc.: 60.94%] [G loss: 1.215621]
451 [D loss: 0.709708, acc.: 59.38%] [G loss: 1.482581]
452 [D loss: 0.699171, acc.: 59.38%] [G loss: 1.691165]
453 [D loss: 0.613617, acc.: 64.06%] [G loss: 0.952431]
454 [D loss: 0.784094, acc.: 51.56%] [G loss: 0.742230]
455 [D loss: 0.548663, acc.: 76.56%] [G loss: 1.161663]
456 [D loss: 0.669638, acc.: 60.94%] [G loss: 1.072115]
457 [D loss: 0.659335, acc.: 60.94%] [G loss: 1.064096]
458 [D loss: 0.737049, acc.: 50.00%] [G loss: 1.417575]
459 [D loss: 0.563953, acc.: 70.31%] [G loss: 1.662233]
460 [D loss: 0.573997, acc.: 65.62%] [G loss: 1.

590 [D loss: 0.824535, acc.: 42.19%] [G loss: 0.973258]
591 [D loss: 0.718522, acc.: 53.12%] [G loss: 1.205039]
592 [D loss: 0.635858, acc.: 60.94%] [G loss: 1.342376]
593 [D loss: 0.588781, acc.: 71.88%] [G loss: 1.178331]
594 [D loss: 0.620314, acc.: 65.62%] [G loss: 1.026330]
595 [D loss: 0.747616, acc.: 50.00%] [G loss: 0.963321]
596 [D loss: 0.644777, acc.: 64.06%] [G loss: 1.375597]
597 [D loss: 0.724878, acc.: 53.12%] [G loss: 1.233688]
598 [D loss: 0.746554, acc.: 53.12%] [G loss: 1.343006]
599 [D loss: 0.780294, acc.: 50.00%] [G loss: 1.053215]
600 [D loss: 0.652195, acc.: 57.81%] [G loss: 1.414881]
601 [D loss: 0.755727, acc.: 50.00%] [G loss: 1.079149]
602 [D loss: 0.770301, acc.: 53.12%] [G loss: 1.055689]
603 [D loss: 0.664509, acc.: 67.19%] [G loss: 1.059531]
604 [D loss: 0.610502, acc.: 59.38%] [G loss: 1.410954]
605 [D loss: 0.577749, acc.: 70.31%] [G loss: 1.402313]
606 [D loss: 0.746319, acc.: 57.81%] [G loss: 1.365547]
607 [D loss: 0.838224, acc.: 43.75%] [G loss: 1.

737 [D loss: 0.633131, acc.: 68.75%] [G loss: 1.186728]
738 [D loss: 0.617721, acc.: 62.50%] [G loss: 1.134958]
739 [D loss: 0.579143, acc.: 71.88%] [G loss: 1.080682]
740 [D loss: 0.661138, acc.: 57.81%] [G loss: 0.970509]
741 [D loss: 0.516466, acc.: 79.69%] [G loss: 1.391673]
742 [D loss: 0.753032, acc.: 53.12%] [G loss: 1.089189]
743 [D loss: 0.571276, acc.: 75.00%] [G loss: 1.109904]
744 [D loss: 0.637394, acc.: 65.62%] [G loss: 1.302953]
745 [D loss: 0.554059, acc.: 75.00%] [G loss: 1.311953]
746 [D loss: 0.588248, acc.: 67.19%] [G loss: 1.138351]
747 [D loss: 0.686019, acc.: 54.69%] [G loss: 1.324281]
748 [D loss: 0.822216, acc.: 45.31%] [G loss: 1.259769]
749 [D loss: 0.743875, acc.: 57.81%] [G loss: 1.355719]
750 [D loss: 0.804996, acc.: 53.12%] [G loss: 1.041138]
751 [D loss: 0.833409, acc.: 45.31%] [G loss: 1.248653]
752 [D loss: 0.636387, acc.: 64.06%] [G loss: 1.155897]
753 [D loss: 0.683590, acc.: 62.50%] [G loss: 1.580840]
754 [D loss: 0.817666, acc.: 50.00%] [G loss: 1.

884 [D loss: 0.528648, acc.: 71.88%] [G loss: 1.373717]
885 [D loss: 0.624374, acc.: 68.75%] [G loss: 1.133422]
886 [D loss: 0.815212, acc.: 53.12%] [G loss: 1.237067]
887 [D loss: 0.706947, acc.: 53.12%] [G loss: 1.143337]
888 [D loss: 0.672889, acc.: 53.12%] [G loss: 1.199716]
889 [D loss: 0.702273, acc.: 60.94%] [G loss: 1.009615]
890 [D loss: 0.766999, acc.: 51.56%] [G loss: 1.059567]
891 [D loss: 0.735122, acc.: 57.81%] [G loss: 1.159432]
892 [D loss: 0.553001, acc.: 68.75%] [G loss: 1.313669]
893 [D loss: 0.805209, acc.: 48.44%] [G loss: 1.052771]
894 [D loss: 0.757396, acc.: 54.69%] [G loss: 0.939153]
895 [D loss: 0.591362, acc.: 64.06%] [G loss: 1.134081]
896 [D loss: 0.619575, acc.: 65.62%] [G loss: 1.298447]
897 [D loss: 0.720061, acc.: 51.56%] [G loss: 1.176075]
898 [D loss: 0.707635, acc.: 50.00%] [G loss: 1.326139]
899 [D loss: 0.664306, acc.: 56.25%] [G loss: 1.081971]
900 [D loss: 0.636299, acc.: 62.50%] [G loss: 1.132024]
901 [D loss: 0.843465, acc.: 48.44%] [G loss: 1.